In [1]:
import torch
from torch.utils.data import DataLoader

import numpy as np

from rpn.build_rpn import RPN_Model
from sam.build_sam import SAM_Model
from transformers import pipeline

from data_builder.build_dataset import PlanetscopeDataset

import os
import time
import json

from eval_utils import filter_boxes, calculate_iou, calculate_precision_recall, filter_masks

import pandas as pd
from rpn import _transforms as T
from torch.utils.data import ConcatDataset

/home/aghosh57/.conda/envs/dl4cv/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
def get_transform(image_enhancement="FALSE"):
    transforms = []
    # converts the image, a PIL image, into a PyTorch Tensor
    if image_enhancement == "TRUE":
        transforms.append(T.ContrastBasedAdaptiveGammaCorrection())
    transforms.append(T.ToTensor())
    return T.Compose(transforms)

#### Read and Print the evaluation config from the JSON file

In [3]:
#Create a folder with unix timestamp
root_path = os.getcwd() + '/results/' + str(int(time.time())) + '/'
os.mkdir(root_path)

with open('eval_config.json', 'r') as f:
    eval_config = json.load(f)


print("Evaluation Configuration")
print(json.dumps(eval_config, indent=1))

Evaluation Configuration
{
 "DATASET1_PATH": "/scratch/aghosh57/SAT-SAM(Dataset)/ps_france/all_dataset/",
 "DATASET1": "FRANCE",
 "DATASET2_PATH": "/scratch/aghosh57/SAT-SAM(Dataset)/ps_france/all_dataset/",
 "DATASET2": "RWANDA",
 "MODEL": "SAM",
 "IMAGE_ENHANCEMENT": "FALSE",
 "FILTRATION": "TRUE",
 "TRAIN_TYPE": "FINETUNE",
 "NMS_THRESHOLD": 0.9,
 "PRED_CONFIDENCE_THRESHOLD": 0.6,
 "ENSEMBLE_BOX_OVERLAP_THRESHOLD": 0.5,
 "ENSEMBLE_BOX_BETA": 0.4,
 "RPN_MODEL_PATH": "rpn/checkpoints/1691966357/rpn_model_1.32.pth",
 "SAM_MODEL_PATH": "sam/checkpoint/sam_vit_l_0b3195.pth"
}


#### Set the device and load the dataset for evaluation

In [4]:
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

dataset1_test = PlanetscopeDataset(eval_config['DATASET1_PATH'], get_transform(image_enhancement=eval_config['IMAGE_ENHANCEMENT']))
dataset2_test = PlanetscopeDataset(eval_config['DATASET2_PATH'], get_transform(image_enhancement=eval_config['IMAGE_ENHANCEMENT']))

dataset_test = ConcatDataset([dataset1_test, dataset2_test])

torch.manual_seed(1)
indices = torch.randperm(len(dataset_test)).tolist()
dataset_test = torch.utils.data.Subset(dataset_test, indices[-int(len(indices)*0.2):])

#### Load the proper model to be used for evaluation

In [5]:
if eval_config['MODEL'] == 'SAT-SAM':
    rpn_model = RPN_Model(eval_config['RPN_MODEL_PATH'], 2, device, eval_config['TRAIN_TYPE'])
    sam_model = SAM_Model(eval_config['SAM_MODEL_PATH'], 'large', device)
elif eval_config['MODEL'] == 'SAM':
    vanilla_sam_model = pipeline("mask-generation", model="facebook/sam-vit-large", device=device)
elif eval_config['MODEL'] == 'MASKRCNN':
    maskrcnn_model = RPN_Model(eval_config['RPN_MODEL_PATH'], 2, device, eval_config['TRAIN_TYPE']) #Load the Pre-Trained MaskRCNN model

In [6]:
results = pd.DataFrame(columns=['parcel_id', 'parcel_path', 'gt_mask_ct', 'pred_mask_ct', 'mean_iou', 'p_50', 'r_50', 'p_70', 'r_70', 'p_90', 'r_90'])

for i, (id, sam_image, rpn_image, target, ensemble, path)  in enumerate(dataset_test): 
    try:
        if eval_config['MODEL'] == 'SAT-SAM':
            print('SAT-SAM')
            rpn_image = rpn_image.squeeze(0).to(device)  
            predictions = rpn_model.predict(rpn_image)
            predictions = rpn_model.postprocess(predictions, nms_threshold=eval_config['NMS_THRESHOLD'], score_threshold=eval_config['PRED_CONFIDENCE_THRESHOLD'])
            
            if eval_config['FILTRATION'] == 'TRUE':
                filtered_predictions = filter_boxes(predictions, ensemble, eval_config['ENSEMBLE_BOX_BETA'], eval_config['ENSEMBLE_BOX_OVERLAP_THRESHOLD'])
                low_res_masks, iou_predictions = sam_model.predict(sam_image, filtered_predictions)
            else:
                low_res_masks, iou_predictions = sam_model.predict(sam_image, predictions['boxes'])
            
            high_res_masks = sam_model.postprocess(low_res_masks, tuple(sam_image.size))
            pred_masks = high_res_masks.squeeze().cpu().numpy()
        
        elif eval_config['MODEL'] == 'SAM':
            print('SAM')
            outputs = vanilla_sam_model(sam_image, points_per_batch=32)
            if eval_config['FILTRATION'] == 'TRUE':
                pred_masks = filter_masks(outputs, ensemble, eval_config['ENSEMBLE_BOX_OVERLAP_THRESHOLD'])
            else:
                pred_masks = outputs["masks"]
        
        elif eval_config['MODEL'] == 'MASKRCNN':
            print('MASKRCNN')
            rpn_image = rpn_image.squeeze(0).to(device)
            predictions = maskrcnn_model.predict(rpn_image)
            predictions = maskrcnn_model.postprocess(predictions, nms_threshold=eval_config['NMS_THRESHOLD'], score_threshold=eval_config['PRED_CONFIDENCE_THRESHOLD'])
            if eval_config['FILTRATION'] == 'TRUE':
                pred_masks = filter_masks(predictions, ensemble, eval_config['ENSEMBLE_BOX_OVERLAP_THRESHOLD'])
            else:
                pred_masks = predictions['masks']

        iou_score, iou_matrix = calculate_iou(target_masks=np.array(target['masks']), predicted_masks=np.array(pred_masks))
        
        p_50, r_50 = calculate_precision_recall(iou_matrix, len(pred_masks), len(target['masks']), threshold=0.5)
        p_70, r_70 = calculate_precision_recall(iou_matrix, len(pred_masks), len(target['masks']), threshold=0.7)
        p_90, r_90 = calculate_precision_recall(iou_matrix, len(pred_masks), len(target['masks']), threshold=0.9)
        
        # Round off the values to 2 decimal places
        iou_score = round(iou_score, 2)
        p_50, r_50 = round(p_50, 2), round(r_50, 2)
        p_70, r_70 = round(p_70, 2), round(r_70, 2)
        p_90, r_90 = round(p_90, 2), round(r_90, 2)

        print("Image Id: ", id, " Average IoU Score: ", iou_score)
        print("IoU Matrix: ", iou_matrix)
        print("Precision: ", p_50, p_70, p_90)
        print("Recall: ", r_50, r_70, r_90)

        results.loc[i] = [id, path, len(target['masks']), len(pred_masks), iou_score, p_50, r_50, p_70, r_70, p_90, r_90]
    except:
        print("Error in image: ", id)
        continue

SAM
num_target_masks:  14
num_predicted_masks:  20
row_ind:  [ 0  1  2  3  4  5  6  7  8  9 10 11 12 13]
col_ind:  [11  8 12 19 16  0  1  4 13 15  3  6  5  2]
Image Id:  2072  Average IoU Score:  0.33
IoU Matrix:  [0.78688745 0.30245475 0.81934847 0.02213466 0.7412176  0.
 0.41461752 0.         0.00114416 0.54875169 0.95510246 0.
 0.         0.04372061]
Precision:  0.25 0.2 0.05
Recall:  0.36 0.29 0.07
SAM
num_target_masks:  13
num_predicted_masks:  43
row_ind:  [ 0  1  2  3  4  5  6  7  8  9 10 11 12]
col_ind:  [ 9 17  2 24  8  1 13  4 16  7 36 10 19]
Image Id:  1306  Average IoU Score:  0.65
IoU Matrix:  [9.32900433e-01 2.05175601e-01 8.92612649e-01 6.95345858e-01
 7.44688735e-01 0.00000000e+00 7.94004107e-01 8.88192729e-01
 4.76084641e-01 8.93612185e-01 5.26038927e-04 9.31718842e-01
 9.58179667e-01]
Precision:  0.21 0.19 0.07
Recall:  0.69 0.62 0.23
SAM
num_target_masks:  16
num_predicted_masks:  14
row_ind:  [ 0  1  2  3  4  5  6  7  8  9 10 11 14 15]
col_ind:  [ 0  5  4 10  6  7  

/home/aghosh57/.conda/envs/dl4cv/lib/python3.9/site-packages/transformers/pipelines/base.py:1090: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(


num_target_masks:  14
num_predicted_masks:  31
row_ind:  [ 0  1  2  3  4  5  6  7  8  9 10 11 12 13]
col_ind:  [ 3  2 25  7 22 10 20 11 12 16 30 15  1 19]
Image Id:  124  Average IoU Score:  0.45
IoU Matrix:  [7.69334439e-01 7.80004661e-01 1.38297872e-01 6.90997567e-01
 1.51649434e-01 4.19986024e-01 8.65273683e-03 8.01061008e-01
 8.94556962e-01 7.87940208e-02 3.45661943e-04 4.86513321e-02
 7.19094602e-01 8.00866635e-01]
Precision:  0.23 0.19 0.0
Recall:  0.5 0.43 0.0
SAM


/home/aghosh57/.conda/envs/dl4cv/lib/python3.9/site-packages/transformers/pipelines/base.py:1090: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(


num_target_masks:  15
num_predicted_masks:  24
row_ind:  [ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14]
col_ind:  [18  9  1 11  6  3  2  8  5  7  4 13 10  0 12]
Image Id:  1411  Average IoU Score:  0.45
IoU Matrix:  [0.69990975 0.63160778 0.92222859 0.         0.83236321 0.88934982
 0.         0.         0.         0.         0.9092219  0.92602448
 0.         0.94635119 0.        ]
Precision:  0.33 0.25 0.17
Recall:  0.53 0.4 0.27
SAM


...

/home/aghosh57/.conda/envs/dl4cv/lib/python3.9/site-packages/transformers/pipelines/base.py:1090: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(


num_target_masks:  42
num_predicted_masks:  69
row_ind:  [ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23
 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41]
col_ind:  [39 28  5 14 12 58 24 19  8 13 10 16 22 18  7  4 26 15 43  9 35 11 25  0
 31 23 34 27  1 66  6 41 17 47 21 59 50 49 33 20  3  2]
Image Id:  317  Average IoU Score:  0.51
IoU Matrix:  [0.40413793 0.         0.81885522 0.85465116 0.88641425 0.76885406
 0.60861691 0.32950283 0.87417678 0.         0.49481912 0.85031759
 0.04977105 0.         0.79711158 0.         0.89615872 0.
 0.65790977 0.         0.70026216 0.         0.77514793 0.83595227
 0.56426464 0.00674068 0.02609335 0.80592287 0.93791108 0.01194673
 0.94744046 0.93275335 0.         0.78280543 0.79214609 0.15027829
 0.15849057 0.89639212 0.44839858 0.35936547 0.93048516 0.97956403]
Precision:  0.33 0.29 0.07
Recall:  0.55 0.48 0.12
SAM


/home/aghosh57/.conda/envs/dl4cv/lib/python3.9/site-packages/transformers/pipelines/base.py:1090: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(


num_target_masks:  28
num_predicted_masks:  0
row_ind:  []
col_ind:  []
Error in image:  5209
SAM
num_target_masks:  1
num_predicted_masks:  3
row_ind:  [0]
col_ind:  [0]
Image Id:  2855  Average IoU Score:  0.79
IoU Matrix:  [0.79033916]
Precision:  0.33 0.33 0.0
Recall:  1.0 1.0 0.0
SAM


/home/aghosh57/.conda/envs/dl4cv/lib/python3.9/site-packages/transformers/pipelines/base.py:1090: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(


In [ ]:
mean_IoU = results['mean_iou'].mean()
eval_config['MEAN_IOU'] = mean_IoU

In [ ]:
results.to_csv(root_path + 'results', index=False)

# save model and configuration
with open(root_path + 'eval_config.json', 'w') as f:
    json.dump(eval_config, f, indent=1)
    f.close()
print("Evaluation Configuration saved to " + root_path + 'train_config.json') 